In [1]:
"""
 * @ Author: Lucas Glasner (lgvivanco96@gmail.com)
 * @ Create Time: 1969-12-31 21:00:00
 * @ Modified by: Lucas Glasner, 
 * @ Modified time: 2024-04-01 17:17:25
 * @ Description:
        Python translation of the "rational method" for computing peak runoff
        on drainage basins.
 */

"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# ----------------------------------- paths ---------------------------------- #
input_parameters    = 'data/parameters.xlsx'          # <- PARAMETER SPREASHEET!!
output_spreadsheet  = 'data/Caudales_MRacional.xlsx'  # <- MODEL OUTPUT !!!

# --------------------------- load basin properties -------------------------- #
parameters_names   = ['basin area', 'basin mean slope', 'H max', 'H min',
                      'H med','basin river length', 'basin centroid length',
                      'curve number', 'moisture condition', 
                      'basin_runoff_coefficient']

parameters         = pd.read_excel(input_parameters,
                                sheet_name='basin_parameters',
                                index_col=0)
parameters         = parameters.loc[parameters_names].drop('units', axis=1)

# ---------------------------- load basin rainfall --------------------------- #
precips_mm24hr   = pd.read_excel('data/parameters.xlsx',
                                sheet_name='precipitation',
                                index_col=0)

In [3]:
# ----------------------- duration coefficient routines ---------------------- #
def grunsky_coef(storm_duration, ref_duration=24):
    """
    This function computes the duration coefficient
    given by the Grunsky Formula.
    
    References: 
        ???

    Args:
        storm_duration (array_like): storm duration in (hours)
        ref_duration (array_like): Reference duration (hours).
            Defaults to 24 hr

    Returns:
        CD (array_like): Duration coefficient in (dimensionless)
    """
    CD = np.sqrt(storm_duration/ref_duration)
    return CD


def bell_coef(storm_duration, ref_duration=24):
    """
    This function computes the duration coefficient
    given by the Bell Formula.
    
    References: 
        Bell, F.C. (1969) Generalized Rainfall-Duration-Frequency
        Relationships. Journal of Hydraulic Division, ASCE, 95, 311-327.  

    Args:
        storm_duration (array_like): duration in (hours)

    Returns:
        CD (array_like): Duration coefficient in (dimensionless)
    """
    a    = (0.54*((storm_duration*60)**0.25)-0.5)
    b    = grunsky_coef(1, ref_duration)
    CD   = a*b
    return CD


def duration_coef(storm_duration,
                  ref_duration=24,
                  bell_threshold=1,
                  duration_threshold=10/60):
    """
    This function is a merge of Grunsky and Bell Formulations
    of the Duration Coefficient. The idea is to use Bell's 
    Formula only when the input duration is less than the "bell_threshold"
    parameter. In addition, when the duration is less than the
    "duration_threshold" the duration is set to the "duration_threshold".

    Args:
        storm_duration (array_like): Storm duration in hours
        bell_threshold (float, optional): Duration threshold for changing
            between Grunsky and Bell formulas. Defaults to 1 (hour).
        duration_threshold (float, optional): Minimum storm duration.
            Defaults to 10 minutes (1/6 hours). 

    Returns:
        coef (array_like): Duration coefficients (dimensionless)
    """
    def is_iterable(obj):
        """
        Simple function to check if an object
        is an iterable.

        Args:
            obj (any): Any python class

        Returns:
            (bool): True if iterable False if not
        """
        try:
            iter(obj)
            return True
        except TypeError:
            return False
    t = storm_duration
    if is_iterable(t):
        coefs = np.empty(len(t))
        for i in range(len(coefs)):
            if t[i]<duration_threshold:
                coefs[i] = duration_coef(duration_threshold, ref_duration)
            elif (t[i]>=duration_threshold) and (t[i]<bell_threshold):
                coefs[i] = bell_coef(t[i], ref_duration)
            else:
                coefs[i] = grunsky_coef(t[i], ref_duration)
    else:
        coefs = duration_coef([t], ref_duration, bell_threshold)
        coefs = coefs.item()
    return coefs

In [4]:
# -------- curve number correction for antecedent moisture conditions -------- #
def CN_correction(CN_II, AMC):
    """
    This function changes the curve number value according to antecedent
    moisture conditions (AMC)...

    Reference: 
        Ven Te Chow (1988), Applied Hydrology. MCGrow-Hill
        Soil Conservation Service, Urban hydrology for small watersheds,
        tech. re/. No. 55, U. S. Dept. of Agriculture, Washington, D.E:., 1975.

    Args:
        CN_II (float): curve number under normal condition
        AMC (str): Antecedent moisture condition.
            Options: 'dry', 'wet' or 'normal'

    Raises:
        RuntimeError: If AMC is different than 'dry', 'wet' or 'normal'

    Returns:
        CN_I or CN_III (float): _description_
    """
    if AMC=='dry':
        CN_I = 4.2*CN_II/(10-0.058*CN_II)
        return CN_I
    elif AMC=='normal':
        return CN_II
    elif AMC=='wet':
        CN_III = 23*CN_II/(10+0.13*CN_II)
        return CN_III
    else:
        text = f'AMC="{AMC}"'
        text = text+' Moisture condition can only be "dry, "neutral" or "wet"'
        raise RuntimeError(text)

In [5]:
# -------------------- Concentration time for rural basins ------------------- #
def tc_SCS(basin_mriverlen_km,basin_meanslope_,curve_number_):
    """
    USA Soil Conservation Service (SCS) method.
    Valid for rural basins ¿?.
    
    Reference:
        ???

    Args:
        basin_mriverlen_km (float): Main river length in (km)
        basin_meanslope_ (float): Basin mean slope in m/m
        curve_number_ (float): Basin curve number (dimensionless)

    Returns:
        Tc (float): Concentration time (minutes)
    """
    a  = ((1000/curve_number_)-9)**0.7
    b  = basin_mriverlen_km**0.8/(basin_meanslope_*100)**0.5
    Tc = 3.42*a*b
    return Tc

def tc_kirpich(basin_mriverlen_km, basin_deltaheights_m):
    """
    Kirpich equation method.
    Valid for small and rural basins ¿?.
    
    Reference:
        ???
    
    Args:
        basin_mriverlen_km (float): Main river length in (km)
        basin_deltaheights_m (float): Difference between minimum and maximum
            basin elevation (m)

    Returns:
        Tc (float): Concentration time (minutes)
    """
    Tc = ((1000*basin_mriverlen_km)**1.15)/(basin_deltaheights_m**0.385)/51
    return Tc

def tc_giandotti(basin_mriverlen_km, basin_meanheight, basin_area_km2):
    """
    Giandotti equation method.
    Valid for small basins (< 20km2) with high slope (>10%) ¿?. 

    Reference:
        ???
    
    Args:
        basin_mriverlen_km (float): Main river length in (km)
        basin_meanheight (float): Basin mean height (meters)
        basin_area_km2 (float): Basin area (km2)

    Returns:
        Tc (float): Concentration time (minutes)
    """
    a  = (4*basin_area_km2**0.5+1.5*basin_mriverlen_km)
    b  = (0.8*basin_meanheight**0.5)
    Tc = 60*a/b
    return Tc

def tc_california(basin_mriverlen_km,basin_deltaheights_m):
    """
    California Culverts Practice (1942) equation.
    Valid for mountain basins ¿?.
    
    Reference: 
        ???
    
    Args:
        basin_mriverlen_km (float): Main river length in (km)
        basin_deltaheights_m (float): Difference between minimum and maximum
            basin elevation (m)
            
    Returns:
        Tc (float): Concentration time (minutes)
    
    """
    Tc=57*(basin_mriverlen_km**3/basin_deltaheights_m)**0.385
    return Tc

def tc_spain(basin_mriverlen_km,basin_meanslope_):
    """
    Equation of Spanish/Spain regulation.

    Reference:
        ???
        
    Args:
        basin_mriverlen_km (float): Main river length in (km)
        basin_meanslope_ (float): Basin mean slope in m/m

    Returns:
        Tc (float): Concentration time (minutes)
    """
    Tc = 18*(basin_mriverlen_km**0.76)/((basin_meanslope_*100)**0.19)
    return Tc

In [6]:
# ------------------------------- main routines ------------------------------ #
def compute_tc(parameters):
    """
    Given the dataframe with basin parameters this function computes
    the concentration time with all the methods and merges them in a single
    table.

    Args:
        parameters (DataFrame): pandas DataFrame with basin parameters

    Returns:
        (DataFrame): Basin concentration times computed with different methods.
    """
    parameters = parameters.copy()
    # SCS concentration time
    new_curve_number_ = [CN_correction(CN, AMC)
                        for AMC,CN in zip(parameters.loc['moisture condition'],
                                        parameters.loc['curve number'])]
    parameters.loc['curve number'] = new_curve_number_
    basin_tc_SCS = tc_SCS(parameters.loc['basin river length'],
                        parameters.loc['basin mean slope'],
                        parameters.loc['curve number'])

    # Kirpich concentration time
    basin_tc_kirpich = tc_kirpich(parameters.loc['basin river length'],
                            parameters.loc['H max']-parameters.loc['H min'])

    # Giandotti concentration time
    basin_tc_giandotti = tc_giandotti(parameters.loc['basin river length'],
                                    parameters.loc['H med'],
                                    parameters.loc['basin area'])

    # California concentration time
    basin_tc_california = tc_california(parameters.loc['basin river length'],
                            parameters.loc['H max']-parameters.loc['H min'])

    # Spanish norms concentration time
    basin_tc_spain = tc_spain(parameters.loc['basin river length'],
                              parameters.loc['basin mean slope'])
    
    basin_tcs = pd.concat([basin_tc_SCS, basin_tc_kirpich, basin_tc_giandotti,
                           basin_tc_california, basin_tc_spain], axis=1)
    basin_tcs.columns = ['tc_SCS', 'tc_kirpich', 'tc_giandotti',
                         'tc_california', 'tc_spain']
    return basin_tcs


def RationalMethod(parameters, precips_mm24hr):
    """
    Given dataframes with basin parameters, precipitations, and frequency factors
    for the runoff coefficient, this function computes the peak runoff of the
    rational method. 

    Args:
        parameters (DataFrame): pandas DataFrame with basin parameters
        precips_mm24hr (DataFrame): pandas DataFrame with precipitation for
            each basin and return period.

    Returns:
        basin_tcs, precips_mmXhr, peakrunoff (tuple):
            DataFrames with concentration time, precipitation intensity at the
            concentration time and model's peak runoff. 
    """
    # Runoff coefficient "frequency factors"
    runoff_FF  = pd.Series([1, 1, 1, 1.1, 1.2, 1.25, 1.275, 1.3],
                        index=[2,5,10,25,50,100,150,200])
    precips_mm24hr = precips_mm24hr.copy()
    parameters     = parameters.copy()
    # Compute concentration time and change units to hours
    basin_tcs = compute_tc(parameters)/60

    # Compute the duration coefficient for the concentration time
    DCoeffs = [duration_coef(basin_tcs.iloc[:,i].values)
                            for i in range(len(basin_tcs.columns))]
    DCoeffs = pd.DataFrame(DCoeffs,
                           columns=basin_tcs.index,
                           index=basin_tcs.columns)
    # Compute precipitation associated with the concentration time duration
    precips_mmXhr = pd.concat([precips_mm24hr*DCoeffs.loc[method]/basin_tcs[method]
                            for method in DCoeffs.index],
                            keys=DCoeffs.index)
    # Compute runoff coefficient for all basins and return periods
    runoff_coefs = [runoff_FF*parameters.loc['basin_runoff_coefficient'].loc[basin]
                    for basin in parameters.columns]
    runoff_coefs = pd.concat(runoff_coefs, keys=parameters.columns).unstack().T

    # Compute peak runoff
    basin_areas  = parameters.loc['basin area']
    peakrunoff   = [runoff_coefs*precips_mmXhr.loc[method]*basin_areas/3.6
                    for method in DCoeffs.index]
    peakrunoff   = pd.concat(peakrunoff,
                            keys=DCoeffs.index)
    return basin_tcs.T, precips_mmXhr, peakrunoff

In [7]:
%%time
# --------------------------------- run model -------------------------------- #
basin_tcs, precips_mmXhr, peakrunoff = RationalMethod(parameters,
                                                      precips_mm24hr)

# ------------------------------- save results ------------------------------- #
with pd.ExcelWriter(output_spreadsheet, mode='w', engine='openpyxl') as writer:
    # Store concentration times in a .xlsx
    basin_tcs.to_excel(writer, sheet_name='concentration_time_hr')
    
with pd.ExcelWriter(output_spreadsheet, mode='a', engine='openpyxl') as writer:
    # Append results in other sheets of the same .xlsx
    precips_mmXhr.to_excel(writer, sheet_name='rainfall_intensity_mmhr-1')
    
with pd.ExcelWriter(output_spreadsheet, mode='a', engine='openpyxl') as writer:
    # Append results in other sheets of the same .xlsx
    peakrunoff.to_excel(writer, sheet_name='peakrunoff_m3s')

CPU times: user 109 ms, sys: 8.76 ms, total: 117 ms
Wall time: 272 ms
